# Whatsapp Export

In [38]:
# import packages
import pandas as pd
import numpy as np

In [39]:
# Read in CSV
df = pd.read_csv('parsed_export.csv',sep="|")
# convert to datetime
df['datetime']=pd.to_datetime(df['datetime'])
df.head()

,datetime,name,message
0,2019-02-21 09:09:00,Joe,Wow what an amazing chat!
1,2019-02-21 09:10:00,Dom,"I agree, it is amazing!"
2,2019-02-21 09:12:00,Joe,You're the best!
3,2019-02-21 09:12:00,Dom,"No, you are the best!"
4,2019-02-21 09:12:00,Dom,<Media omitted>


# Censoring
In the interest of privacy I would suggest removing any personal details. The below code allows you to censor names, locations, ideas and whatever else you like.

In [41]:
# Censor name and words
# Names list - CHANGE THESE to include any personal names to remove.
names = { 
            'old_name' : 'new_name',
            'Joe' : 'J',
            'Dom' : 'D', 
}

# Censor list - CHANGE THESE to include any words you want to hide, I'd suggest including the above names.
censor_list = [
    ['word_to_be_censored','censored_word'],
    ['best','good'],
    ['amazing','alright'],
]

# Replace messages
def censor(message):
    for censor in censor_list:
        message = message.replace(censor[0],censor[1])
    
    return message

# remove any spaces
df['name'] = df['name'].str.strip()
# Replace names
df = df.replace(
    {
        'name' : names
    }
)

# censor all messages
df['message'] = df['message'].apply(censor)
df.head()

,datetime,name,message
0,2019-02-21 09:09:00,J,Wow what an alright chat!
1,2019-02-21 09:10:00,D,"I agree, it is alright!"
2,2019-02-21 09:12:00,J,You're the good!
3,2019-02-21 09:12:00,D,"No, you are the good!"
4,2019-02-21 09:12:00,D,<Media omitted>


# Gotchas
Every Media message has been replaced with "<Media Omitted>", we need to remove these or they might be our most popular words.

Capitalized words will show up twice too, so "Joe" and "joe" might both show up in our word cloud. To avoid this we can capitalize, or lowercase everything.

In [42]:
# Drop rows with media messages
df = df[df['message'] != '<Media omitted>']

# Capitalize everything
df['message'] = df['message'].apply(lambda x: x.title())
df.head()

,datetime,name,message
0,2019-02-21 09:09:00,J,Wow What An Alright Chat!
1,2019-02-21 09:10:00,D,"I Agree, It Is Alright!"
2,2019-02-21 09:12:00,J,You'Re The Good!
3,2019-02-21 09:12:00,D,"No, You Are The Good!"
